In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import csv

# 1. 取得數據

In [2]:
player = requests.get('http://www.cpbl.com.tw/players/person.html?player_id=B0K0&teamno=B04')
if player.status_code == 200:
    print('Connection OK!')
    soup = BeautifulSoup(player.text,'html.parser')
    player_info = soup.find('div', class_ = 'player_info')
    player_score_pitching = soup.find_all('table', class_ = 'std_tb')

    
    #print(player_score_pitching.text)
#print(player_score_pitching)

Connection OK!


# 2.爬取基本資料

In [3]:
# player_info_list = []
# p = {}
# player_name = player_info.find('div', class_ = 'player_info_name')
# p.update({'球員姓名':player_name.text[:3]})
# p.update({'球員背號:':player_name.text[4:6]})
# player_team = player_name.find('span')
# # print('球員姓名:',player_name.text[:3])
# # print('球員背號:',player_name.text[4:6])
# # print(player_team.text[0:2],":",player_team.text[3:5])
# player_detail = player_info.find_all('tr')
# for i in player_detail:
#     k = i.find_all('td')
#     p.update({k[0].text[0:2]:k[0].text[3:]})
#     p.update({k[1].text[0:2]:k[1].text[3:]})
#     p.update({k[2].text[0:2]:k[2].text[3:]})
#     p.update({k[3].text[0:2]:k[3].text[3:]})
# #     print(k[0].text[0:2],':',k[0].text[3:])
# #     print(k[1].text[0:2],':',k[1].text[3:])
# #     print(k[2].text[0:2],':',k[2].text[3:])
# #     print(k[3].text[0:2],':',k[3].text[3:])
# #print (p)
# player_info_list.append(p)

In [4]:
#爬取球員基本資料
def crawl_player_info(player_info):
    player_info_list = []
    p = {}
    player_name = player_info.find('div', class_ = 'player_info_name')
    p.update({'球員姓名':player_name.text[:3]})
    p.update({'球員背號:':player_name.text[4:6]})
    player_team = player_name.find('span')
    player_detail = player_info.find_all('tr')
    for i in player_detail:
        k = i.find_all('td')
        p.update({k[0].text[0:2]:k[0].text[3:]})
        p.update({k[1].text[0:2]:k[1].text[3:]})
        p.update({k[2].text[0:2]:k[2].text[3:]})
        p.update({k[3].text[0:2]:k[3].text[3:]})
    player_info_list.append(p)
    return player_info_list

# 3.爬取球員成績

In [5]:
# score = player_score_pitching.find_all('tr')
# bar = score[0].find_all('th')
# bar_list = []
# bar_title = []
# player_score = []
# player_score_c = []
# for i in bar:
#     bar_list.append(i.text)
#     bar_title.append(i.get('title'))
# bar_dic = dict(zip(bar_list, bar_title))
# for k in range(1, len(score)):
#     year_score = {}
#     year_score_c = {}
#     each_year = score[k].find_all('td')
#     for i in range(len(each_year)):
#         title = bar_list[i]
#         c_title = bar_title[i]
#         static = each_year[i].text.replace("\n","").replace("\t","").replace("\r","")
#         year_score.update({title:static})
#         year_score_c.update({c_title:static})
#     player_score.append(year_score)
#     player_score_c.append(year_score_c)
# #print(player_score)
# #print(player_score_c)

In [10]:
#爬投手成績
def crawl_player_score(player_score_pitching):
    score = player_score_pitching.find_all('tr')
    bar = score[0].find_all('th')
    bar_list = []
    bar_title = []
    player_score = []
    player_score_c = []
    for i in bar:
        bar_list.append(i.text)
        bar_title.append(i.get('title'))
    bar_dic = dict(zip(bar_list, bar_title))
    for k in range(1, len(score)):
        year_score = {}
        year_score_c = {}
        each_year = score[k].find_all('td')
        for i in range(len(each_year)):
            title = bar_list[i]
            c_title = bar_title[i]
            static = each_year[i].text.replace("\n","").replace("\t","").replace("\r","")
            year_score.update({title:static})
            year_score_c.update({c_title:static})
        player_score.append(year_score)
        player_score_c.append(year_score_c)
    return player_score
    #print(player_score_c)
pitching = crawl_player_score(player_score_pitching[0])
defense = crawl_player_score(player_score_pitching[1])
verus_team = crawl_player_score(player_score_pitching[-2])
highest = crawl_player_score(player_score_pitching[-1])

In [11]:
crawl_player_info(player_info)
pitching = crawl_player_score(player_score_pitching[0])
defense = crawl_player_score(player_score_pitching[1])
verus_team = crawl_player_score(player_score_pitching[-2])
highest = crawl_player_score(player_score_pitching[-1])

In [15]:
def crawl_player_score_csv(player_score_pitching):
    c = open("test.csv","w")
    writer = csv.writer(c)
    score = player_score_pitching.find_all('tr')
    bar = score[0].find_all('th')
    bar_list = []
    bar_title = []
    player_score = []
    player_score_c = []
    for i in bar:
        bar_list.append(i.text)
        bar_title.append(i.get('title'))
    writer.writerow(bar_list)
    writer.writerow(bar_title)
    for k in range(1, len(score)):
        year_score = {}
        year_score_c = {}
        each_year = score[k].find_all('td')
        for i in range(len(each_year)):
            title = bar_list[i]
            c_title = bar_title[i]
            static = each_year[i].text.replace("\n","").replace("\t","").replace("\r","")
            year_score.update({title:static})
            year_score_c.update({c_title:static})
        player_score.append(year_score)
        player_score_c.append(year_score_c)
    return player_score

In [16]:
crawl_player_score_csv(player_score_pitching[0])

[{'YEAR': '2017',
  'TEAM': '富邦',
  'G': '1',
  'GS': '1',
  'GR': '0',
  'CG': '0',
  'SHO': '0',
  'NBB': '0',
  'W': '0',
  'L': '1',
  'SV': '0',
  'BS': '0',
  'HLD': '0',
  'IP': '2.0',
  'WHIP': '4.00',
  'ERA': '13.50',
  'BF': '14',
  'NP': '59',
  'H': '3',
  'HR': '0',
  'BB': '5',
  'IBB': '0',
  'HBP': '0',
  'SO': '0',
  'WP': '1',
  'BK': '0',
  'R': '5',
  'ER': '3',
  'GO': '3',
  'AO': '3',
  'G/F': '1.000'},
 {'YEAR': '2018',
  'TEAM': '富邦',
  'G': '7',
  'GS': '6',
  'GR': '1',
  'CG': '0',
  'SHO': '0',
  'NBB': '0',
  'W': '0',
  'L': '2',
  'SV': '0',
  'BS': '1',
  'HLD': '0',
  'IP': '27.1',
  'WHIP': '1.79',
  'ERA': '7.24',
  'BF': '129',
  'NP': '476',
  'H': '38',
  'HR': '4',
  'BB': '11',
  'IBB': '0',
  'HBP': '2',
  'SO': '13',
  'WP': '3',
  'BK': '0',
  'R': '27',
  'ER': '22',
  'GO': '39',
  'AO': '26',
  'G/F': '1.500'},
 {'YEAR': '2019',
  'TEAM': '富邦',
  'G': '23',
  'GS': '23',
  'GR': '0',
  'CG': '0',
  'SHO': '0',
  'NBB': '0',
  'W': '11',
 